# ETL Pipeline - Data Extraction

This notebook is part of the ETL (Extract, Transform, Load) pipeline for the Capstone 1 project at Purwadhika School. The focus here is on the **extraction phase**, where multiple CSV files from a specified directory are read, filtered, and converted into pandas DataFrames for further processing.

## Objectives
- Read CSV files from the directory specified in `EXTRACT_PATH`.
- Filter files based on specific key columns (`Country Name`, `Country Code`, `Indicator Name`, `Indicator Code`).
- Detect file encodings using `charset_normalizer` to ensure proper reading of CSV files.
- Extract relevant CSV files into pandas DataFrames.

## Libraries Used
- **glob**: To find all CSV files in the specified directory.
- **pandas**: To handle data manipulation and create DataFrames.
- **charset_normalizer**: To detect the encoding of CSV files for accurate reading.

In [16]:
import glob
import pandas as pd
import charset_normalizer

## Configuration and Utility Imports

The following cell imports the `EXTRACT_PATH` from the `settings.py` file in the `config` directory, which specifies the directory containing the input CSV files. The `extract_csv` function from the `extract` module is also imported to handle CSV file extraction.

In [6]:
from config.settings import EXTRACT_PATH
from extract.extract import extract_csv

## Filtering CSV Files

This section identifies CSV files in the `EXTRACT_PATH` directory that contain specific key columns (`Country Name`, `Country Code`, `Indicator Name`, `Indicator Code`). It reads the first 10 lines of each file to check for these columns in the header (assumed to be on line 5). Files that match the criteria are stored in the `filtered_files` list.

In [ ]:
files = glob.glob(f'{EXTRACT_PATH}/*.csv')

key_columns = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
filtered_files = []

for file in files:
    with open(file) as f:
        for i in range(10):
            line = f.readline()
            if i == 4:  # Check line 5 for key columns
                if any(e in line for e in key_columns):
                    filtered_files.append(file)

## Extracting Data to DataFrames

The `extract_data_to_df` function extracts data from CSV files in the specified path into pandas DataFrames. It:
- Uses `glob` to find all CSV files.
- Detects the encoding of each file using `charset_normalizer` to ensure proper reading.
- Checks for the presence of key columns in the specified line (default is line 0, but can be adjusted with `skip_line`).
- Uses the `extract_csv` function to read valid CSV files into DataFrames.
- Returns a list of DataFrames for further processing.

In [ ]:
def extract_data_to_df(path: str, key_columns: list[str], skip_line: int = 0) -> list[pd.DataFrame]:  
    files = glob.glob(f'{path}/*.csv')

    df_list: list[pd.DataFrame] = []

    for file in files:
        with open(file, 'rb') as fil:
            result = charset_normalizer.detect(fil.read())
            print(file)
            print(result)
            
        with open(file) as f:
            for i in range(10):
                line = f.readline()
                if i == skip_line:
                    if any(e in line for e in key_columns):
                        df = extract_csv(file, skip_line)
                        df_list.append(df)
    return df_list

## Running the Extraction

This cell executes the `extract_data_to_df` function to process CSV files in the `EXTRACT_PATH` directory. It checks for the key columns on line 5 (index 4) and prints the detected encoding for each file. The resulting DataFrames are stored in the `df` variable.

In [ ]:
df = extract_data_to_df(EXTRACT_PATH, key_columns, 4)

./data\electricity_access_percent.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\projects_data.csv
{'encoding': 'utf-8', 'language': 'English', 'confidence': 0.917}
./data\mystery.csv
{'encoding': 'UTF-16', 'language': 'Vietnamese', 'confidence': 1.0}
./data\population_data.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\rural_population_percent.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\gdp_data.csv
{'encoding': 'UTF-8-SIG', 'language': 'Finnish', 'confidence': 1.0}


## Verifying File Encodings

This section verifies the encoding of the filtered CSV files using `charset_normalizer`. It reads each file in binary mode, detects the encoding, and prints the results. This step ensures that files are read correctly in the ETL pipeline.

In [18]:
for file in filtered_files:
    with open(file, 'rb') as fil:
        result = charset_normalizer.detect(fil.read())
        print(result)

{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'Finnish', 'confidence': 1.0}


## Notes
- The `extract_csv` function (imported from `extract.extract`) is to handle the actual CSV reading logic with proper encoding and skip rows.
- The `skip_line` parameter in `extract_data_to_df` allows flexibility in handling CSV files with headers in different rows.
- Ensure the `EXTRACT_PATH` in `settings.py` points to the correct directory containing the CSV files.
- The pipeline filters out files like `mystery.csv` if they do not contain the key columns or have incompatible encodings (e.g., UTF-16).